In [1]:
import numpy as np
import os
import json
import cv2 as cv
import shutil
import random

### Check initial 

In [2]:
# paddle anot
# path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/example_paddle_seg/optic_disc_seg/Annotations/H0002.png"


# anot our
# path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_resized/Annotations/GX010026_73.png"
path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_512_0123/Annotations/GX010026_73.png"


img_np = cv.imread(path_img)
# img_np[220:270,220:270,:]
img_np[:,:,:3]

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]]], dtype=uint8)

In [3]:
img_np.shape

(512, 512, 3)

In [4]:
img_np

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]]], dtype=uint8)

In [ ]:
img_np[420:470,420:470,:]

In [ ]:
np.unique(img_np)

## Set labeled imgs

In [5]:
selected_images={"GX010034_14.jpg",
                 "GX010040_19.jpg",
                 "GX010034_10.jpg",
                 "GX010035_50.jpg",
                 "GX010044_4.jpg",
                 "GX010038_1.jpg",
                 "GX010045_15.jpg",
                 "GX010046_6.jpg",
                 "GX010045_17.jpg",
                 "GX010044_6.jpg",
                 "GX010045_17.jpg",
                 "GX010044_6.jpg",
                 "GX010040_31.jpg",
                 "GX010038_17.jpg",
                 "GX010045_29.jpg",
                 "GX010044_5.jpg",
                 "GX010040_3.jpg",
                 "GX010042_1.jpg",
                 "GX010032_8.jpg"}

path_image="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/002_video_data_customer/002_labeled/labeled_Olga_Gelya_21_11_23/img_selected_labeled/"

## Load labeled LS anot 

In [6]:
# pth_name = "project-13-at-2023-11-21-11-39-a223ddf0.json"
pth_name ="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/002_video_data_customer/002_labeled/labeled_Olga_Gelya_21_11_23/project-13-at-2023-11-21-11-39-a223ddf0.json"

with open(pth_name) as user_file:
    parsed_json = json.load(user_file)


# Select just labeled agree with set
need_labels= []

for n in parsed_json:
    if os.path.basename(n['data']['image']) in selected_images:
        need_labels.append(n)

In [7]:
dict_clas={"leaf":1,
            "stem":2,
            "tomato":3}

## Pipe convert LS to paddle_paddle_sef

In [11]:
def relative2absalute(np_arr, resolution_orig):
    np_arr_abs = np.copy(np_arr)
    np_arr_abs[:,:1] = np_arr_abs[:,:1]/100*resolution_orig[1]
    np_arr_abs[:,1:] = np_arr_abs[:,1:]/100*resolution_orig[0]
    # np_arr_abs=np.rint(np_arr_abs).astype(np.int32)
    np_arr_abs=np.rint(np_arr_abs).astype(np.int32)

    return np_arr_abs

def get_resolution(one_imgObj_LS):
    w_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_width']
    h_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_height']
    resolution_orig = (h_orig,w_orig)
    return resolution_orig

def change_ext(name_file, new_ext):
    pre, ext = os.path.splitext(name_file)
    return f"{pre}.{new_ext}"

def LS2paddle_seg(one_imgObj_LS, dict_clas):
    
    resolution_orig=get_resolution(one_imgObj_LS)
    full_mask = np.zeros(shape=resolution_orig, dtype=np.uint32)
    
    
    for result in one_imgObj_LS["annotations"][0]["result"]:
        points=result["value"]['points']
        np_polyg=np.array( [np.array(i) for i in points] )
        # Convert relative points to absolute
        np_polyg_abs=relative2absalute(np_polyg, resolution_orig)
        # Add dimension polyg
        np_polyg_abs=np_polyg_abs[np.newaxis, ... ] 
        # Create temporaty mask
        mask = np.zeros(shape=resolution_orig, dtype=np.uint8)
        
        
        # we can't use here number more than 255 fot this cv.fillPoly functione there fore we fill value=1, then will replace it to => init_ids
        name_label = result["value"]["polygonlabels"][0]
        color_clas=dict_clas[name_label]
        
        filled_mask = cv.fillPoly(mask, pts=np_polyg_abs, color=color_clas)
        filled_mask.max()
    
        full_mask[filled_mask == color_clas] = color_clas
    
    
    # zer_np=  np.zeros(shape=resolution_orig, dtype=np.uint32)
    # # meaning labeled layer in 3d RGB array is last = full_mask
    # result_np = np.stack((zer_np, zer_np, full_mask), axis=-1, dtype=np.uint8)
    print(full_mask.shape)
    full_mask=full_mask.astype(np.uint8)
    return full_mask  # result_np


## pipe to paddle_seg

In [17]:
# dict_clas={ "leaf":80,
#             "stem":160,
#             "tomato":240 }

dict_clas={"leaf":1,
            "stem":2,
            "tomato":3}

dst="/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_gray/Annotations_gray/"

for one_imgObj_LS in need_labels:
    result_np = LS2paddle_seg(one_imgObj_LS, dict_clas)
    # naming path 
    name_image = os.path.basename(one_imgObj_LS['data']['image'])
    name_image= change_ext(name_image, "png")
    dst_path = os.path.join(dst,name_image)

    # save img
    cv.imwrite(dst_path, result_np)

(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)


# SECOND SET

### another paddle_seg

In [18]:
selected_images={"GX010027_42.jpg",
                 "GX010027_1.jpg",
                 "GX010028_70.jpg",
                 "GX010020_89.jpg",
                 "GX010027_6.jpg",
                 "GX010017_20.jpg",
                 "GX010023_34.jpg",
                 "GX010027_46.jpg",
                 "GX010018_63.jpg",
                 "GX010026_12.jpg",
                 "GX010024_4.jpg",
                 "GX010025_60.jpg",
                 "GX010028_20.jpg",
                 "GX010018_9.jpg",
                 "GX010021_34.jpg", # from this point new labels
                 "GX010027_37.jpg",
                 "GX010019_6.jpg",
                 "GX010023_13.jpg",
                 "GX010020_43.jpg",
                 "GX010023_55.jpg",
                 "GX010019_85.jpg",
                 "GX010026_73.jpg",
                 "GX010027_24.jpg",
                 "GX010027_19.jpg",
                 "GX010020_10.jpg",
                 "GX010028_60.jpg",
                 "GX010027_28.jpg"
                 }

In [19]:
pth_name ="/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data_customer/003_labels/002_labeled_studio_LEAF_TOMATO_15.11.23/project-8-at-2023-11-15-17-09-e7686567.json"

with open(pth_name) as user_file:
    parsed_json = json.load(user_file)


# Select just labeled agree with set
need_labels= []

for n in parsed_json:
    if os.path.basename(n['data']['image']) in selected_images:
        need_labels.append(n)

In [20]:
len(need_labels)+17

44

In [21]:
# dict_clas={ "leaf":80,
#             "stem":160,
#             "tomato":240 }

dict_clas={"leaf":1,
            "stem":2,
            "tomato":3}

dst="/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_gray/Annotations_gray/"

for one_imgObj_LS in need_labels:
    result_np = LS2paddle_seg(one_imgObj_LS, dict_clas)
    # naming path 
    name_image = os.path.basename(one_imgObj_LS['data']['image'])
    name_image= change_ext(name_image, "png")
    dst_path = os.path.join(dst,name_image)

    # save img
    cv.imwrite(dst_path, result_np)

(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)
(1520, 2704)


## split to traind and val

In [ ]:
def del_recursiv_DS_Store(path):
    if os.path.isdir(path):
        # Clear file counter
        i = 0
        
        # Runs through all files in the directory
        for root, sub, files in os.walk(path):
            
            for file in files:
        
                # Checks if exists .DS_Store file
                if file == ".DS_Store":
        
                    # Get full path of current .DS_Store file
                    fullpath = os.path.abspath(os.path.join(root, file))
                    print (f"Deleting  + {fullpath}")
        
                    # Remove file
                    os.remove(fullpath)
                    i += 1
        
        print (f"str(i) +  files deleted")

In [ ]:
pth = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/"

del_recursiv_DS_Store(pth)

In [ ]:
pth_labes = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/Annotations"

list_anot = os.listdir(pth_labes)
list_anot
# UpdatedList = random.sample(list_anot, 3)

In [ ]:
def split_TrainVal(l,p):
    amount_for_train = int(len(l)*p)
    train_set = set(random.sample(l, amount_for_train))
    # val_set = list(set(l)-set(l_train))
    l=set(l)
    val_set=l.difference(train_set)
    return list(train_set), list(val_set)

In [ ]:
train_list, val_list = split_TrainVal(list_anot, 0.85)


print(f"Amount train_list : {len(train_list)}")
print(f"Amount val_list   : {len(val_list)}")

print( f"Intersection is none: {set(train_list).intersection( set(val_list))==set()} " )

In [ ]:
def write_anot(exmpl_list, dst_dirout, name_file):
    val_str= ""
    
    img_foldName = "JPEGImages"
    anot_foldName = "Annotations"
    
    for name_file_anot in exmpl_list:
        name_file_jpg = change_ext(name_file_anot, "jpg")
        tmp_str = f"{img_foldName}/{name_file_jpg} {anot_foldName}/{name_file_anot}"
        val_str= f"{val_str}\n{tmp_str}"
    
    val_str=val_str.strip("\n")

    dst_dirout = os.path.join(dst_dirout, name_file)
    with open(dst_dirout, 'w') as f:
        f.write(val_str)
    print(f"Well done!!!")
    return val_str

In [ ]:
name_file = "val_list.txt"
dst_dirout = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/"

exmpl_list = val_list

val_str= write_anot(exmpl_list, dst_dirout, name_file)

In [ ]:
val_str.strip("\n")

In [ ]:
name_file = "train_list.txt"
dst_dirout = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/"

exmpl_list = train_list

val_str= write_anot(exmpl_list, dst_dirout, name_file)

In [ ]:
path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_resized/Annotations/GX010018_9.png"

img_np = cv.imread(path_img)
# img_np[220:270,220:270,:]
img_np[:,:,:3].max()

np.unique(img_np)

In [ ]:
import cv2

# path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/example_paddle_seg/optic_disc_seg/Annotations/H0002.png"
path_img = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_resized/Annotations/GX010018_9.png"



img_np = cv2.imread(path_img, cv2.IMREAD_UNCHANGED)


img_np

In [ ]:
def resize (src, dst, res):
    res = cv.resize(img, (width, height), interpolation = cv.INTER_NEAREST) 
    cv.imwrite

In [ ]:
src = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/Annotations"

dst = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_resized/Annotations"

res =  (512,512)

In [ ]:
import os 

src = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/Annotations"
dst = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_512_0123/Annotations"
res =  (512,512)

for img_name in os.listdir(src):
    src_ful_name = os.path.join(src, img_name)
    dst_ful_name = os.path.join(dst, img_name)

    img_np = cv.imread(src_ful_name)
    img_np = cv.resize(img_np, res, interpolation = cv.INTER_NEAREST)
    # cv.imwrite()
    cv.imwrite(dst_ful_name, img_np) 

In [ ]:
import os 

src = "//Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle/JPEGImages"

dst = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/004_train_Mask2Former_panoptic_23_10_23/paddle_paddle_resized/JPEGImages"

res =  (512,512)
for img_name in os.listdir(src):
    src_ful_name = os.path.join(src, img_name)
    dst_ful_name = os.path.join(dst, img_name)

    img_np = cv.imread(src_ful_name)
    img_np = cv.resize(img_np, res, interpolation = cv.INTER_CUBIC)
    # cv.imwrite()
    cv.imwrite(dst_ful_name, img_np) 